In [1]:
import os

import cv2
import math
import numpy as np
import moviepy 
import torch

from matplotlib import pyplot as plt



from my_alphaction.config import cfg
from my_alphaction.modeling.detector import build_detection_model
from my_alphaction.utils.checkpoint import ActionCheckpointer
from my_alphaction.utils.comm import get_world_size


from my_utils.gen_utils import create_experiment_folder
from my_utils.video_processing import get_video_info, get_frame_from_video
from my_utils.slicing import get_slice_bboxes, generate_sliding_window_gif

from my_utils.video_processing import segment_crop_video
from my_utils.my_ava_preprocessing import ava_preprocessing_cv2, clip_constructor, prepare_collated_batches
from my_utils.ava_postprocessing import concatenate_results
from my_utils.visualization import action_visualizer_frame_index

from my_utils.gen_utils import parse_label_file

from my_utils.ava_postprocessing import clip_boxes_tensor, map_bbox_from_prep_to_crop, map_bbox_from_crop_to_orig


from tqdm import tqdm

### 1. CONFIG
#### 1.1 Main Parameters

In [2]:
model_name = 'VMAEv2'


person_threshold = 0.6 # confidence threshold on actor. 0.6 is the defualt
sampling_rate = 3 # sampling rate: 4 is the defualt
top_k = 5 # number of actions per person
video_path = '../input_dir/markt2_fight.mp4'

slice_height = 800
slice_width = 1000
overlap_ratio = 0.1

starting_frame_index = 100
length_input = 200

exp_dict = {'model_name': model_name,
            'model_params': {'person_threshold': person_threshold, 
                             'sampling_rate': sampling_rate},
            'orig_post_processing':{'top_k': top_k},
            'aggregation': {'method': {}, 
                            'params': {}},
            'video_path': video_path,
            'slicing_params': {'slice_height': slice_height, 
                               'slice_width': slice_width, 
                               'overlap_ratio':overlap_ratio},
            'video_params': {'st_frame_index': starting_frame_index, 
                             'length_input':length_input
                             }
           }



In [3]:
if model_name == 'VMAEv2':
    config_file = '../config_files/VMAEv2-ViTB-16x4.yaml'
if model_name == 'VMAE':
    config_file = '../config_files/VMAE-ViTB-16x4.yaml'


In [4]:
cfg.merge_from_file(config_file)

In [5]:
# change model weight path
if model_name == 'VMAEv2':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAEv2_ViTB_16x4.pth"])
if model_name == 'VMAE':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAE_ViTB_16x4.pth"])

# change output dir
cfg.merge_from_list(["OUTPUT_DIR", "../output_dir/"])

# change person threshold
cfg.merge_from_list(["MODEL.STM.PERSON_THRESHOLD", person_threshold])

# change sampling rate
cfg.merge_from_list(["DATA.SAMPLING_RATE", sampling_rate])

# change path for data_dir
cfg.merge_from_list(["DATA.PATH_TO_DATA_DIR", "/work/ava"])

# folder name of annotations
cfg.merge_from_list(["AVA.ANNOTATION_DIR", "annotations/"])

# file name of  frame_lists
cfg.merge_from_list(["AVA.TRAIN_LISTS", ['sample.csv']])
cfg.merge_from_list(["AVA.TEST_LISTS", ['sample.csv']])

# file name of predicted_bboxes
cfg.merge_from_list(["AVA.TRAIN_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])
cfg.merge_from_list(["AVA.TEST_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])

# file name of exlusions
cfg.merge_from_list(["AVA.EXCLUSION_FILE", 'ava_sample_train_excluded_timestamps_v2.2.csv'])

# number of batches in test scenario
cfg.merge_from_list(["TEST.VIDEOS_PER_BATCH", 1])

# number of workers
cfg.merge_from_list(["DATALOADER.NUM_WORKERS", 1])


In [6]:
cfg.ViT.USE_CHECKPOINT

True

In [7]:
cfg.merge_from_list(["ViT.USE_CHECKPOINT", False])

In [8]:
cfg.ViT.USE_CHECKPOINT

False

In [9]:
debug = True
if debug:
    # The shape of model input should be divisible into this. Otherwise, padding 0 to left and bottum. 
    print("cfg.DATALOADER.SIZE_DIVISIBILITY: ", cfg.DATALOADER.SIZE_DIVISIBILITY)
    
    # Sampling rate in constructing the clips.
    self_sample_rate =  cfg.DATA.SAMPLING_RATE
    print("cfg.DATA.SAMPLING_RATE: ", cfg.DATA.SAMPLING_RATE)
    
    # Length of clip
    self_video_length = cfg.DATA.NUM_FRAMES
    print("cfg.DATA.NUM_FRAMES: ", cfg.DATA.NUM_FRAMES)
    
    # Length of sequence frames from which a clip is constructed.
    self_seq_len = self_video_length * self_sample_rate
    print("self_seq_len: ", self_seq_len)
    
    self_num_classes = cfg.MODEL.STM.ACTION_CLASSES
    print("cfg.MODEL.STM.ACTION_CLASSES: ", self_num_classes)
    
    # Augmentation params.
    self_data_mean = cfg.DATA.MEAN
    self_data_std = cfg.DATA.STD
    self_use_bgr = cfg.AVA.BGR
    print("Augmentation params: ", self_data_mean, self_data_std, self_use_bgr)
    
    self_jitter_min_scale = cfg.DATA.TEST_MIN_SCALES
    self_jitter_max_scale = cfg.DATA.TEST_MAX_SCALE
    self_test_force_flip = cfg.AVA.TEST_FORCE_FLIP

    print("scale and flip params", self_jitter_min_scale, self_jitter_max_scale, self_test_force_flip)

cfg.DATALOADER.SIZE_DIVISIBILITY:  32
cfg.DATA.SAMPLING_RATE:  3
cfg.DATA.NUM_FRAMES:  16
self_seq_len:  48
cfg.MODEL.STM.ACTION_CLASSES:  80
Augmentation params:  [0.45, 0.45, 0.45] [0.225, 0.225, 0.225] False
scale and flip params [256] 1333 False


In [10]:
model = build_detection_model(cfg)

In [11]:
checkpointer = ActionCheckpointer(cfg, model, save_dir="../output_dir/")
checkpointer.load(cfg.MODEL.WEIGHT)

{}

In [12]:
if False:
    backbone_module = model._modules['backbone']
    backbone_module = backbone_module.eval()

In [13]:
import torch.onnx
import torch

In [14]:
import onnx


In [16]:
slow_video = torch.randn(1, 3, 16, 256, 320).to('cuda')
values = [[320., 256., 320., 256.]]

# Create the tensor
whwh = torch.tensor(values, device='cuda')

model = model.eval().to('cuda')

# Export the model to ONNX with static batch size
torch.onnx.export(model, 
                  args = slow_video, 
                  f="whole_model_dyn_batch.onnx", 
                  input_names=["slow_video"], 
                  output_names=["output"],
                  opset_version=17,
                  verbose=False
                  #dynamic_axes={"slow_video": {0: "batch_size"}, "whwh": {0: "batch_size"}, "output": {0: "batch_size"}}
                 )

SymbolicValueError: Failed to export a node '%2645 : Long(device=cpu) = onnx::Squeeze(%2643, %2644), scope: my_alphaction.modeling.detector.stm_detector.STMDetector::/my_alphaction.modeling.stm_decoder.stm_decoder.STMDecoder::stm_head/my_alphaction.modeling.stm_decoder.stm_decoder.AMStage::decoder_stages.0/my_alphaction.modeling.stm_decoder.stm_decoder.AdaptiveSTSamplingMixing::samplingmixing/my_alphaction.modeling.stm_decoder.util.adaptive_mixing_operator.AdaptiveMixing::adaptive_mixing_s # /work/my_alphaction/modeling/stm_decoder/util/adaptive_mixing_operator.py:78:0
' (in list node %2654 : int[] = prim::ListConstruct(%2645, %2653), scope: my_alphaction.modeling.detector.stm_detector.STMDetector::/my_alphaction.modeling.stm_decoder.stm_decoder.STMDecoder::stm_head/my_alphaction.modeling.stm_decoder.stm_decoder.AMStage::decoder_stages.0/my_alphaction.modeling.stm_decoder.stm_decoder.AdaptiveSTSamplingMixing::samplingmixing/my_alphaction.modeling.stm_decoder.util.adaptive_mixing_operat

In [ ]:
traced_model = torch.jit.trace(model, [slow_video])

In [ ]:
with torch.jit.optimized_execution(True):
    scripted_model = torch.jit.script(model, [slow_video]).to("cuda")

In [ ]:
torch.onnx.export(model, slow_video, "model_v2.onnx", do_constant_folding=False, verbose=True)

In [ ]:
torch.onnx.export(traced_model, slow_video, "traced_model.onnx", verbose=True)

In [ ]:
my_model = onnx.load("your_model.onnx")
onnx.checker.check_model(my_model)

In [ ]:

# iterate through inputs of the graph
for input in my_model.graph.input:
    print (input.name, end=": ")
    # get type of input tensor
    tensor_type = input.type.tensor_type
    # check if it has a shape:
    if (tensor_type.HasField("shape")):
        # iterate through dimensions of the shape:
        for d in tensor_type.shape.dim:
            # the dimension may have a definite (integer) value or a symbolic identifier or neither:
            if (d.HasField("dim_value")):
                print (d.dim_value, end=", ")  # known dimension
            elif (d.HasField("dim_param")):
                print (d.dim_param, end=", ")  # unknown dimension with symbolic name
            else:
                print ("?", end=", ")  # unknown dimension with no name
    else:
        print ("unknown rank", end="")
    print()

In [ ]:
# Extract backbone and lateral_convs modules

import torch.nn as nn

backbone_module = model._modules['backbone']
lateral_convs_module = model._modules['lateral_convs']

# Assuming you want to combine them into a single model
class CombinedModel(nn.Module):
    def __init__(self, backbone, lateral_convs):
        super(CombinedModel, self).__init__()
        self.backbone = backbone
        self.lateral_convs = lateral_convs

    def forward(self, x):
        # Forward pass through the backbone and lateral_convs
        backbone_output = self.backbone(x)
        lateral_convs_output = self.lateral_convs(backbone_output)
        return lateral_convs_output

# Create the combined model instance
combined_model = CombinedModel(backbone_module, lateral_convs_module)

In [ ]:
example_input = torch.randn(1, 3, 16, 256, 320)


In [ ]:
combined_model([example_input])

In [ ]:
model

In [ ]:
import onnx
print("ONNX version:", onnx.__version__)

In [ ]:
from typing import Any, Sequence
import numpy as np
import onnx
import onnxruntime
import subprocess
import sys
from copy import deepcopy
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np

import onnx
from onnx.backend.test.case.test_case import TestCase
from onnx.backend.test.case.utils import import_recursive
from onnx.onnx_pb import (
    AttributeProto,
    FunctionProto,
    GraphProto,
    ModelProto,
    NodeProto,
    TensorProto,
    TypeProto,
)

_NodeTestCases = []
_TargetOpType = None
_DiffOpTypes = None


def _extract_value_info(
    input: Union[List[Any], np.ndarray, None],
    name: str,
    type_proto: Optional[TypeProto] = None,
) -> onnx.ValueInfoProto:
    if type_proto is None:
        if input is None:
            raise NotImplementedError(
                "_extract_value_info: both input and type_proto arguments cannot be None."
            )
        elif isinstance(input, list):
            elem_type = onnx.helper.np_dtype_to_tensor_dtype(input[0].dtype)
            shape = None
            tensor_type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)
            type_proto = onnx.helper.make_sequence_type_proto(tensor_type_proto)
        elif isinstance(input, TensorProto):
            elem_type = input.data_type
            shape = tuple(input.dims)
            type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)
        else:
            elem_type = onnx.helper.np_dtype_to_tensor_dtype(input.dtype)
            shape = input.shape
            type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)

    return onnx.helper.make_value_info(name, type_proto)

def expect(
    node: onnx.NodeProto,
    inputs: Sequence[np.ndarray],
    outputs: Sequence[np.ndarray],
    name: str,
    **kwargs: Any,
) -> None:
    # Builds the model
    present_inputs = [x for x in node.input if (x != "")]
    present_outputs = [x for x in node.output if (x != "")]
    input_type_protos = [None] * len(inputs)
    if "input_type_protos" in kwargs:
        input_type_protos = kwargs["input_type_protos"]
        del kwargs["input_type_protos"]
    output_type_protos = [None] * len(outputs)
    if "output_type_protos" in kwargs:
        output_type_protos = kwargs["output_type_protos"]
        del kwargs["output_type_protos"]
    inputs_vi = [
        _extract_value_info(arr, arr_name, input_type)
        for arr, arr_name, input_type in zip(inputs, present_inputs, input_type_protos)
    ]
    outputs_vi = [
        _extract_value_info(arr, arr_name, output_type)
        for arr, arr_name, output_type in zip(
            outputs, present_outputs, output_type_protos
        )
    ]
    graph = onnx.helper.make_graph(
        nodes=[node], name=name, inputs=inputs_vi, outputs=outputs_vi
    )
    kwargs["producer_name"] = "backend-test"

    if "opset_imports" not in kwargs:
        # To make sure the model will be produced with the same opset_version after opset changes
        # By default, it uses since_version as opset_version for produced models
        produce_opset_version = onnx.defs.get_schema(
            node.op_type, domain=node.domain
        ).since_version
        kwargs["opset_imports"] = [
            onnx.helper.make_operatorsetid(node.domain, produce_opset_version)
        ]

    model = onnx.helper.make_model_gen_version(graph, **kwargs)

    # Checking the produces are the expected ones.
    sess = onnxruntime.InferenceSession(model.SerializeToString(),
                                        providers=["CPUExecutionProvider"])
    feeds = {name: value for name, value in zip(node.input, inputs)}
    results = sess.run(None, feeds)
    for expected, output in zip(outputs, results):
        return (results, outputs)
        return np.testing.assert_allclose(expected, output)

In [ ]:
node = onnx.helper.make_node(
    "Squeeze",
    inputs=["x", "axes"],
    outputs=["y"],
)
x = np.random.randn(1, 3, 4, 5).astype(np.float32)
axes = np.array([0], dtype=np.int64)
y = np.squeeze(x, axis=0)

s = expect(node, inputs=[x, axes], outputs=[y], name="test_squeeze")

In [ ]:
s

In [ ]:
import numpy as np
import onnx
import onnxruntime as ort

# Step 1: Define the ONNX node and create an ONNX graph
node = onnx.helper.make_node(
    "Squeeze",
    inputs=["x", "axes"],
    outputs=["y"],
)
graph = onnx.helper.make_graph([node], "squeeze_graph", inputs=[onnx.helper.make_tensor_value_info("x", onnx.TensorProto.FLOAT, (1, 3, 4, 5)), onnx.helper.make_tensor_value_info("axes", onnx.TensorProto.INT64, (1,))], outputs=[onnx.helper.make_tensor_value_info("y", onnx.TensorProto.FLOAT, None)])

# Step 2: Create an ONNX model with the graph
onnx_model = onnx.helper.make_model(graph)

# Save the ONNX model to a file
onnx.save(onnx_model, "squeeze_model.onnx")

# Step 3: Load the ONNX model with ONNX Runtime
sess = ort.InferenceSession("squeeze_model.onnx")

# Step 4: Prepare input data
x = np.random.randn(1, 3, 4, 5).astype(np.float32)
axes = np.array([0], dtype=np.int64)

# Step 5: Run inference
output = sess.run(["y"], {"x": x, "axes": axes})

# Print the output
print("Output:", output)

In [ ]:
output[0].shape

In [ ]:
axes

In [ ]:
# Step 1: Define the ONNX node and create an ONNX graph
node = onnx.helper.make_node(
    "Squeeze",
    inputs=["x"],
    outputs=["y"],
    axes=[0, 1]  # Squeeze along axes 0 and 1
)
graph = onnx.helper.make_graph([node], "squeeze_graph", inputs=[onnx.helper.make_tensor_value_info("x", onnx.TensorProto.FLOAT, (1, 3, 4, 5))], outputs=[onnx.helper.make_tensor_value_info("y", onnx.TensorProto.FLOAT, None)])

# Step 2: Create an ONNX model with the graph
onnx_model = onnx.helper.make_model(graph)

# Save the ONNX model to a file
onnx.save(onnx_model, "squeeze_model.onnx")

# Step 3: Load the ONNX model with ONNX Runtime
sess = ort.InferenceSession("squeeze_model.onnx")

# Step 4: Prepare input data
x = np.random.randn(1, 3, 4, 5).astype(np.float32)

# Step 5: Run inference
output = sess.run(["y"], {"x": x})

# Print the output
print("Output:", output)